<a href="https://colab.research.google.com/github/ciepielajan/GeoFabrik_Data_Extract/blob/main/GEOfabrik_analiza_zawartosci.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import folium
import pandas as pd
from plotnine import *
!pip -q install geopandas
from geopandas import GeoDataFrame

%matplotlib inline

     |████████████████████████████████| 1.0MB 7.7MB/s 
     |████████████████████████████████| 6.5MB 22.6MB/s 
     |████████████████████████████████| 14.8MB 273kB/s 


Load data from shapefile downloaded from http://download.geofabrik.de/

In [10]:
!wget "http://download.geofabrik.de/europe/poland/malopolskie-latest-free.shp.zip"  -O map.shp.zip

--2021-04-04 09:32:59--  http://download.geofabrik.de/europe/poland/malopolskie-latest-free.shp.zip
Resolving download.geofabrik.de (download.geofabrik.de)... 88.99.142.44, 116.202.112.212, 138.201.219.183
Connecting to download.geofabrik.de (download.geofabrik.de)|88.99.142.44|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 232211549 (221M) [application/zip]
Saving to: ‘map.shp.zip’

map.shp.zip         100%[===================>] 221.45M  19.8MB/s    in 12s     

2021-04-04 09:33:12 (18.2 MB/s) - ‘map.shp.zip’ saved [232211549/232211549]



In [11]:
!unzip -q map.shp.zip -d /content/maps

In [ ]:
from os import listdir

files_dir =  listdir("/content/maps")
files_list = []
# files_name_list = []
for names in files_dir:
    if names.endswith(".shp"):
        files_list.append(names)
        # files_name_list.append(names.split("osm_")[1].split("_free")[0])

files_list = sorted(files_list)
# files_name_list = sorted(files_name_list)
print("ilość plików", len(files_list))
for name in files_list:
  print(name)

In [117]:
# fclass_lists = []
# for name in files_list:
#   fclass_frame = GeoDataFrame.from_file(f'maps/{name}', encoding='utf-8').fclass.value_counts().index
#   for x in fclass_frame:
#      fclass_lists.append(x)

In [121]:
!pip -q install textblob
from textblob import TextBlob
import textblob.exceptions # obsługa błędu jeżli brak tłumaczenia

In [ ]:
# fclass_lists_PL = []
# for class_eng in fclass_lists:
#   try:
#     translate = TextBlob(class_eng).translate(from_lang='en', to="pl")
#   except textblob.exceptions.NotTranslated:
#    translate = class_eng

#   fclass_lists_PL.append(str(translate))

In [ ]:
# translate_eng_pol = { eng:pl for eng, pl in zip(fclass_lists,fclass_lists_PL)}
# translate_eng_pol

In [149]:
def translate_to_PL(word):
  try:
    translate = TextBlob(word).translate(from_lang='en', to="pl")
  except textblob.exceptions.NotTranslated:
    translate = word

  return str(translate)

In [55]:
%time
roads = GeoDataFrame.from_file('maps/gis_osm_roads_free_1.shp', encoding='utf-8')
print(roads.shape)
roads.head()

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.77 µs
(372670, 11)


,osm_id,code,fclass,name,ref,oneway,maxspeed,layer,bridge,tunnel,geometry
0,2954554,5122,residential,Ignacego Mościckiego,None,B,40,0,F,F,"LINESTRING (20.03436 50.08068, 20.03452 50.080..."
1,2954556,5115,tertiary,Stefana Żeromskiego,None,B,50,0,F,F,"LINESTRING (20.03605 50.07939, 20.03626 50.079..."
2,2954557,5122,residential,None,None,B,30,0,F,F,"LINESTRING (20.03550 50.07982, 20.03566 50.079..."
3,2954558,5122,residential,None,None,B,30,0,F,F,"LINESTRING (20.03484 50.08193, 20.03516 50.081..."
4,3989492,5124,pedestrian,Floriańska,None,F,20,0,F,F,"LINESTRING (19.94134 50.06483, 19.94133 50.064..."


In [ ]:
fclass_unique_trans = {}
for word in roads.fclass.value_counts().index:
  fclass_unique_trans[word] = translate_to_PL(word)

In [144]:
roads["fclass_PL"] = roads["fclass"].apply(lambda x: translate_to_PL(x))

HTTPError: ignored

In [ ]:
roads.fclass_PL = roads.fclass.replace(fclass_unique_trans)

In [ ]:
roads.head()

In [56]:
roads.fclass.value_counts()

service           135908
footway            58960
track              53057
residential        46681
path               15268
tertiary           12248
unclassified        7736
primary             5409
secondary           5385
steps               5327
track_grade5        4268
cycleway            4019
track_grade1        3947
track_grade4        3502
track_grade3        3317
track_grade2        2806
living_street       2197
pedestrian          1004
motorway             540
motorway_link        293
primary_link         248
trunk                180
tertiary_link        147
secondary_link       101
trunk_link            68
bridleway             54
Name: fclass, dtype: int64

In [151]:
roads.maxspeed.value_counts().sort_index()

0      347782
5          43
10         93
15         18
20       2017
25          6
30       3987
40       2131
50      11407
60        414
70       1843
80         53
90       1878
100       315
110         1
120       128
130         1
140       553
Name: maxspeed, dtype: int64

In [57]:
%time
pofw = GeoDataFrame.from_file('maps/gis_osm_pofw_free_1.shp', encoding='utf-8')
print(pofw.shape)
pofw.head()

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.25 µs
(1718, 5)


,osm_id,code,fclass,name,geometry
0,672084028,3100,christian,None,POINT (20.71168 49.41291)
1,672084039,3100,christian,None,POINT (20.72726 49.41456)
2,672084043,3100,christian,None,POINT (20.72149 49.41717)
3,672084046,3100,christian,None,POINT (20.69901 49.41636)
4,672084110,3100,christian,None,POINT (20.79427 49.33344)


In [58]:
pofw.fclass.value_counts()

christian               1335
christian_catholic       356
christian_orthodox        10
jewish                    10
christian_protestant       4
christian_methodist        1
christian_lutheran         1
muslim                     1
Name: fclass, dtype: int64

In [59]:
%time
waterways = GeoDataFrame.from_file('maps/gis_osm_waterways_free_1.shp', encoding='utf-8')
print(waterways.shape)
waterways.head()

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.01 µs
(46600, 6)


,osm_id,code,fclass,width,name,geometry
0,5819083,8101,river,0,Biała,"LINESTRING (21.06901 49.44025, 21.06736 49.440..."
1,5819592,8101,river,0,Dunajec,"LINESTRING (20.68030 49.80995, 20.68049 49.810..."
2,5909314,8101,river,0,Ropa,"LINESTRING (21.04776 49.60824, 21.04990 49.609..."
3,5909428,8101,river,0,Dunajec,"LINESTRING (20.65635 49.69135, 20.66124 49.693..."
4,5909630,8101,river,0,Dunajec,"LINESTRING (20.47315 49.53902, 20.47866 49.537..."


In [60]:
waterways.fclass.value_counts()

stream    43521
river      1582
drain      1300
canal       197
Name: fclass, dtype: int64

In [61]:
%time
railways = GeoDataFrame.from_file('maps/gis_osm_railways_free_1.shp', encoding='utf-8')
print(railways.shape)
railways.head()

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.96 µs
(6123, 8)


,osm_id,code,fclass,name,layer,bridge,tunnel,geometry
0,4820685,6101,rail,None,0,F,F,"LINESTRING (19.81982 50.10854, 19.83200 50.099..."
1,5901391,6101,rail,None,0,F,F,"LINESTRING (20.15380 50.02779, 20.15478 50.030..."
2,5901548,6101,rail,None,0,F,F,"LINESTRING (19.96535 50.04225, 19.96580 50.041..."
3,5901574,6101,rail,None,0,F,F,"LINESTRING (20.37067 49.98760, 20.38054 49.98492)"
4,5901827,6101,rail,None,1,T,F,"LINESTRING (20.15380 50.02779, 20.15373 50.02761)"


In [62]:
railways.fclass.value_counts()

rail            5150
tram             769
narrow_gauge     178
monorail          19
funicular          7
Name: fclass, dtype: int64

In [63]:
%time
transport_a = GeoDataFrame.from_file('maps/gis_osm_transport_a_free_1.shp', encoding='utf-8')
print(transport_a.shape)
transport_a.head()

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.87 µs
(638, 5)


,osm_id,code,fclass,name,geometry
0,19796272,5656,apron,None,"POLYGON ((19.78131 50.07318, 19.78147 50.07336..."
1,26424475,5656,apron,None,"POLYGON ((20.19613 50.08498, 20.19929 50.08502..."
2,39272551,5622,bus_station,Azory,"POLYGON ((19.90876 50.08659, 19.90877 50.08663..."
3,43457434,5622,bus_station,Dworzec Autobusowy,"POLYGON ((20.69748 49.61713, 20.69845 49.61749..."
4,49156766,5621,bus_stop,Prokocim Szpital,"POLYGON ((20.00093 50.01492, 20.00142 50.01518..."


In [64]:
transport_a.fclass.value_counts()

bus_stop           523
helipad             42
bus_station         35
taxi                16
apron                9
airport              8
tram_stop            2
railway_station      2
airfield             1
Name: fclass, dtype: int64

In [65]:
%time
pois = GeoDataFrame.from_file('maps/gis_osm_pois_free_1.shp', encoding='utf-8')
print(pois.shape)
pois.head()

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.58 µs
(33833, 5)


,osm_id,code,fclass,name,geometry
0,25606107,2701,tourist_info,Rysy,POINT (20.08808 49.17946)
1,26860250,2422,camp_site,Camp-Cup,POINT (19.62601 49.57885)
2,26860273,2422,camp_site,Harenda,POINT (19.98500 49.32417)
3,26860287,2422,camp_site,Krakowianka,POINT (19.92453 50.01552)
4,31944307,2404,guesthouse,Wawrzka,POINT (20.98333 49.58333)


In [66]:
pois.fclass.value_counts()

bench             5008
convenience       2033
tourist_info      1726
wayside_shrine    1605
restaurant        1481
                  ... 
battlefield          2
embassy              1
prison               1
lighthouse           1
water_mill           1
Name: fclass, Length: 132, dtype: int64

In [67]:
%time
transport = GeoDataFrame.from_file('maps/gis_osm_transport_free_1.shp', encoding='utf-8')
print(transport.shape)
transport.head()

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.34 µs
(10106, 5)


,osm_id,code,fclass,name,geometry
0,41943557,5601,railway_station,Kraków Mydlniki,POINT (19.85970 50.08360)
1,48267476,5601,railway_station,Bochnia,POINT (20.43151 49.97729)
2,48522023,5601,railway_station,Kraków Batowice,POINT (19.99547 50.10753)
3,48523754,5601,railway_station,Kraków Nowa Huta,POINT (20.12950 50.09058)
4,48526714,5602,railway_halt,Słomniki Miasto,POINT (20.07637 50.23587)


In [68]:
transport.fclass.value_counts()

bus_stop           9391
tram_stop           365
railway_halt        122
railway_station      91
taxi                 79
bus_station          30
ferry_terminal       21
helipad               6
airport               1
Name: fclass, dtype: int64

In [69]:
%time
places_a = GeoDataFrame.from_file('maps/gis_osm_places_a_free_1.shp', encoding='utf-8')
print(places_a.shape)
places_a.head()

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.01 µs
(34, 6)


,osm_id,code,fclass,population,name,geometry
0,69553535,1050,locality,0,Hala Krupowa,"POLYGON ((19.65468 49.62424, 19.65506 49.62446..."
1,130583808,1020,island,0,None,"POLYGON ((20.07783 50.25781, 20.07798 50.25802..."
2,168403564,1020,island,0,Ptasia Wyspa,"POLYGON ((20.24106 49.45946, 20.24108 49.45953..."
3,3288260,1003,village,0,Książniczki,"POLYGON ((20.00059 50.13281, 20.00079 50.13293..."
4,259975428,1050,locality,0,Czoło,"POLYGON ((20.23166 49.52170, 20.23166 49.52193..."


In [70]:
places_a.fclass.value_counts()

island      15
locality     9
suburb       8
farm         1
village      1
Name: fclass, dtype: int64

In [72]:
%time
traffic = GeoDataFrame.from_file('maps/gis_osm_traffic_free_1.shp', encoding='utf-8')
print(traffic.shape)
traffic.head()

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.34 µs
(30034, 5)


,osm_id,code,fclass,name,geometry
0,29782991,5201,traffic_signals,None,POINT (20.03122 50.07678)
1,29782993,5204,crossing,None,POINT (20.02799 50.07927)
2,29782996,5201,traffic_signals,None,POINT (20.02795 50.07986)
3,30375883,5204,crossing,None,POINT (19.94247 50.06449)
4,30376964,5201,traffic_signals,None,POINT (19.90412 50.04919)


In [71]:
traffic.fclass.value_counts()

crossing               16718
street_lamp             6422
parking_bicycle         2294
parking                 1372
traffic_signals         1289
stop                     544
turning_circle           436
fuel                     392
weir                     280
motorway_junction         63
waterfall                 52
speed_camera              43
parking_underground       25
marina                    24
slipway                   22
mini_roundabout           20
dam                       20
lock_gate                 14
pier                       2
parking_multistorey        2
Name: fclass, dtype: int64

In [73]:
%time
places_a = GeoDataFrame.from_file('maps/gis_osm_places_a_free_1.shp', encoding='utf-8')
print(places_a.shape)
places_a.head()

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.44 µs
(34, 6)


,osm_id,code,fclass,population,name,geometry
0,69553535,1050,locality,0,Hala Krupowa,"POLYGON ((19.65468 49.62424, 19.65506 49.62446..."
1,130583808,1020,island,0,None,"POLYGON ((20.07783 50.25781, 20.07798 50.25802..."
2,168403564,1020,island,0,Ptasia Wyspa,"POLYGON ((20.24106 49.45946, 20.24108 49.45953..."
3,3288260,1003,village,0,Książniczki,"POLYGON ((20.00059 50.13281, 20.00079 50.13293..."
4,259975428,1050,locality,0,Czoło,"POLYGON ((20.23166 49.52170, 20.23166 49.52193..."


In [74]:
places_a.fclass.value_counts()

island      15
locality     9
suburb       8
farm         1
village      1
Name: fclass, dtype: int64

In [75]:
%time
natural_a = GeoDataFrame.from_file('maps/gis_osm_natural_a_free_1.shp', encoding='utf-8')
print(natural_a.shape)
natural_a.head()

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.48 µs
(110, 5)


,osm_id,code,fclass,name,geometry
0,25156249,4141,beach,kąpielisko strzeżone,"POLYGON ((19.79176 50.04785, 19.79182 50.04787..."
1,25156251,4141,beach,None,"POLYGON ((19.79706 50.04863, 19.79706 50.04875..."
2,25157073,4141,beach,Kryspinów Plaża Naturystyczna,"POLYGON ((19.78679 50.05572, 19.78715 50.05571..."
3,25450792,4141,beach,None,"POLYGON ((19.44993 50.13863, 19.44999 50.13883..."
4,25450793,4141,beach,None,"POLYGON ((19.45208 50.13762, 19.45223 50.13770..."


In [76]:
natural_a.fclass.value_counts()

beach      75
cliff      21
glacier    12
spring      2
Name: fclass, dtype: int64

In [77]:
%time
landuse_a = GeoDataFrame.from_file('maps/gis_osm_landuse_a_free_1.shp', encoding='utf-8')
print(landuse_a.shape)
landuse_a.head()

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.96 µs
(182184, 5)


,osm_id,code,fclass,name,geometry
0,20190259,7201,forest,None,"POLYGON ((19.80347 50.10261, 19.80353 50.10273..."
1,20190402,7201,forest,None,"POLYGON ((19.80738 50.09817, 19.80747 50.09828..."
2,20190511,7201,forest,None,"POLYGON ((19.82236 50.09557, 19.82337 50.09550..."
3,20193458,7218,grass,None,"POLYGON ((20.05270 50.07950, 20.05278 50.07960..."
4,22938927,7201,forest,None,"POLYGON ((19.84609 50.01414, 19.84612 50.01432..."


In [78]:
landuse_a.fclass.value_counts()

forest               49396
residential          32975
farmland             27604
scrub                21636
meadow               17930
grass                16404
orchard               7339
farmyard              2344
industrial            1818
cemetery              1372
retail                 899
commercial             686
park                   679
allotments             514
recreation_ground      175
nature_reserve         155
quarry                 151
vineyard                49
military                31
heath                   27
Name: fclass, dtype: int64

In [79]:
%time
traffic_a = GeoDataFrame.from_file('maps/gis_osm_traffic_a_free_1.shp', encoding='utf-8')
print(traffic_a.shape)
traffic_a.head()

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.39 µs
(11724, 5)


,osm_id,code,fclass,name,geometry
0,22787855,5260,parking,None,"POLYGON ((19.65908 49.25882, 19.65908 49.25910..."
1,23068881,5260,parking,None,"POLYGON ((20.43310 49.97005, 20.43326 49.97006..."
2,25042802,5332,weir,Stopień Wodny Przewóz,"POLYGON ((20.09762 50.04161, 20.09763 50.04163..."
3,25091288,5260,parking,None,"POLYGON ((21.16171 49.65080, 21.16187 49.65093..."
4,25123245,5260,parking,None,"POLYGON ((19.93655 50.06769, 19.93739 50.06780..."


In [80]:
traffic_a.fclass.value_counts()

parking                11193
fuel                     253
parking_bicycle          108
pier                      67
dam                       46
parking_multistorey       22
marina                    11
service                   11
parking_underground       10
slipway                    2
weir                       1
Name: fclass, dtype: int64

In [81]:
%time
pois_a = GeoDataFrame.from_file('maps/gis_osm_pois_a_free_1.shp', encoding='utf-8')
print(pois_a.shape)
pois_a.head()

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.87 µs
(16096, 5)


,osm_id,code,fclass,name,geometry
0,19796313,2401,hotel,Hilton Garden Inn Krakow Airport,"POLYGON ((19.80211 50.07165, 19.80322 50.07180..."
1,19803351,2252,pitch,None,"POLYGON ((20.05971 50.06508, 20.06070 50.06514..."
2,19841872,2252,pitch,None,"POLYGON ((19.90818 50.06320, 19.90840 50.06376..."
3,19842194,2258,track,None,"POLYGON ((19.91531 50.07965, 19.91535 50.07977..."
4,23062643,2204,park,None,"POLYGON ((20.42388 49.96833, 20.42399 49.96837..."


In [82]:
pois_a.fclass.value_counts()

pitch             3482
graveyard         1372
playground        1322
school            1081
shelter            704
                  ... 
golf_course          2
drinking_water       2
windmill             2
car_rental           1
outdoor_shop         1
Name: fclass, Length: 120, dtype: int64

In [83]:
%time
places = GeoDataFrame.from_file('maps/gis_osm_places_free_1.shp', encoding='utf-8')
print(places.shape)
places.head()

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.63 µs
(18002, 6)


,osm_id,code,fclass,population,name,geometry
0,26035799,1003,village,364,Osturňa,POINT (20.23333 49.33333)
1,26036002,1004,hamlet,139,Legnava,POINT (20.84981 49.33675)
2,26036012,1003,village,446,Malý Lipník,POINT (20.79956 49.33732)
3,26036092,1003,village,361,Veľká Franková,POINT (20.29345 49.33804)
4,26036193,1003,village,1319,Suchá Hora,POINT (19.79011 49.36631)


In [84]:
places.fclass.value_counts()

hamlet      14849
village      1976
locality     1065
town           62
suburb         45
city            3
island          1
farm            1
Name: fclass, dtype: int64

In [85]:
%time
buildings_a = GeoDataFrame.from_file('maps/gis_osm_buildings_a_free_1.shp', encoding='utf-8')
print(buildings_a.shape)
buildings_a.head()

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 7.87 µs
(1349151, 6)


,osm_id,code,fclass,name,type,geometry
0,19796313,1500,building,Hilton Garden Inn Krakow Airport,hotel,"POLYGON ((19.80211 50.07165, 19.80322 50.07180..."
1,23062762,1500,building,Kościół pw. Świętego Mikołaja Biskupa,church,"POLYGON ((20.42797 49.96935, 20.42801 49.96936..."
2,23068834,1500,building,Dzwonnica drewniana XV w.,None,"POLYGON ((20.42812 49.96904, 20.42823 49.96908..."
3,23256528,1500,building,Sukiennice,public,"POLYGON ((19.93679 50.06135, 19.93681 50.06137..."
4,24208843,1500,building,"Główna łaźnia ""Sauna""",None,"POLYGON ((19.16641 50.03883, 19.16643 50.03947..."


In [86]:
buildings_a.fclass.value_counts()

building    1349151
Name: fclass, dtype: int64

In [87]:
%time
natural = GeoDataFrame.from_file('maps/gis_osm_natural_free_1.shp', encoding='utf-8')
print(natural.shape)
natural.head()

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.39 µs
(51119, 5)


,osm_id,code,fclass,name,geometry
0,26864278,4111,peak,Pilsko,POINT (19.31675 49.52800)
1,26864983,4111,peak,Turbacz,POINT (20.11138 49.54282)
2,31943589,4111,peak,Dziamera,POINT (21.30261 49.55284)
3,226153423,4111,peak,Kopiec Kościuszki,POINT (19.89335 50.05492)
4,273454953,4121,tree,None,POINT (19.91568 50.06385)


In [88]:
natural.fclass.value_counts()

tree             48124
peak              2323
cave_entrance      415
spring             238
cliff               18
beach                1
Name: fclass, dtype: int64

In [89]:
%time
water_a = GeoDataFrame.from_file('maps/gis_osm_water_a_free_1.shp', encoding='utf-8')
print(water_a.shape)
water_a.head()

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.72 µs
(8792, 5)


,osm_id,code,fclass,name,geometry
0,5909396,8201,reservoir,Jezioro Klimkowskie,"POLYGON ((21.07813 49.55540, 21.07818 49.55546..."
1,5909432,8202,river,Dunajec,"POLYGON ((20.66969 49.60742, 20.67004 49.60778..."
2,7628519,8200,water,None,"POLYGON ((20.64521 49.79566, 20.64539 49.79588..."
3,19796883,8200,water,None,"POLYGON ((19.81997 50.08815, 19.82002 50.08827..."
4,19796886,8200,water,None,"POLYGON ((19.82254 50.09034, 19.82400 50.09111..."


In [90]:
water_a.fclass.value_counts()

water        6712
wetland       981
reservoir     734
riverbank     231
river         120
glacier        12
dock            2
Name: fclass, dtype: int64

In [91]:
# m = folium.Map([50.05,19.95], zoom_start=13,
#                tiles="cartodbpositron"
#                )

# folium.GeoJson(water_a).add_to(m)

# m

In [92]:
# m = folium.Map([50.05,19.95], zoom_start=13,
#                tiles="cartodbpositron"
#                )

# folium.GeoJson(natural).add_to(m)

# m